In [1]:
import numpy as np
import scipy as sp
from pyscf import gto, scf, ao2mo

# Background

Hartree-Fock Self-Consistent Field Theory (HF-SCF) with restricted orbitals and closed-shell systems (RHF) sovles the following pseduo eigenvalue problem:

$$FC = S C \epsilon$$

called the Roothan equations.

This is soolved self-consistently for orbital coefficient matrix $C$ and orbital energy eigenvalues $\epsilon$

The Fock matrix has elements $F_{\mu\nu}$ in the **atomic orbital basis** as :

$$F_{\mu\nu} = H_{\mu\nu} + 2 (\mu\nu|\lambda \sigma)D_{\lambda \sigma} -  (\mu\lambda|\nu \sigma)D_{\lambda \sigma}$$

where

$$D_{\lambda \sigma} = C_{\sigma i} C_{\lambda i}$$

Note $C$ is an $(N \times M)$ matrix, where $N$ is no. of **atomic basis** fucntions and $M$ the number of molecular orbitals

Note too that it is common to write Coulomb and Exchange matrices J and K, with elements (here square brackets denote their dependents on the density matrix!):

$$J[D_{\lambda \sigma}]_{\mu\nu} = (\mu\nu|\lambda \sigma)D_{\lambda \sigma}$$

$$K[D_{\lambda \sigma}]_{\mu\nu} = (\mu\lambda|\nu \sigma)D_{\lambda \sigma}$$

The Fock matrix can therefore be found by:

$$F = H + 2J -  K$$

# Manual Calculation!

### 1. Define molecule in PySCF

In [2]:
geometry = """
O
H 1 1.1
H 1 1.1 2 104
"""
basis =  'cc-pvdz'

In [3]:
# geometry = """
# H 0 0 0 
# H 0 0 0.74
# """
# basis =  'STO-3G'

In [4]:


full_system_mol = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol.build()

### 2. Compute static 1e- and 2e- integrals with PySCF

In [5]:
S = full_system_mol.intor('int1e_ovlp') # 1e- electron overlap <i|j>

eri_ao = full_system_mol.intor('int2e') # 2e- electron repulsion integrals in AO basis <ij|kl>

In [6]:
n_docc = full_system_mol.nelectron // 2 # number of double occupied orbitals
n_bas_ft = full_system_mol.nao

print(f'Number of occupied orbitals:{n_docc:4.0f}')
print(f'Number of basis functions: {n_bas_ft:6.0f}')

Number of occupied orbitals:   5
Number of basis functions:     24


### 3. Build Core Hamiltonian

In [7]:
T = full_system_mol.intor('int1e_kin')
V = full_system_mol.intor('int1e_nuc')

H_core = T+V

In [8]:
# check H_core calculated is the SAME!
np.allclose(H_core, scf.hf.get_hcore(full_system_mol))

True

### 4. Check roothan and S matrix

We can solve:

$$FC = S C \epsilon$$

if $S$ is the identity matrix... however:

In [9]:
test = np.allclose(S, np.eye(S.shape[0]))
print(f'is S the identity matrix?: {test}')
if test is False:
    print()
    print('AO basis is not orthonormal')

is S the identity matrix?: False

AO basis is not orthonormal


Overall we cannot ignore the AO overlap matrix... $F$ cannot simply be diagonlized to solve for the orbital coefficent matrix!

LUCKILY: we can overcome this issue, by **transforming** the AO basis so that all the basis functions are orthonormal!

aka we need a transform:

$$U^{\dagger} S U = \mathcal{I}$$

Clearly this makes S diagonal!

But how do we do this? (pg 143-144 Szabo and https://www.youtube.com/watch?v=2N104Nf-_L4)

1. Symmetric orthogonalization
    - uses the symmetric inverse square root of the overlap matrix
    - $ U = S^{-\frac{1}{2}}$

as clearly:

$$U^{\dagger} S U = S^{-\frac{1}{2}} S S^{-\frac{1}{2}}  = S^{-\frac{1}{2}} S^{+\frac{1}{2}} = \mathcal{I}$$

**IMPORTANT numerical problems can occur if the overlap matrix has small eigenvalues, which may occur for large systems or for systems where diffuse basis sets are used**

2. canonical orthogonalization
    - uses the symmetric inverse square root of the overlap matrix and additional unitary $V$
    - V is the unitary we use to diagonlize the matrix $S$
        - we then truncate according to the eignvalues of $V$... for small terms there will be numerical problems so they are REMOVED (TRUNCATED)
    - $ U = V S^{-\frac{1}{2}}$
    
This numerical problem may be avoided by using canonical orthogonalization, in which an asymmetric inverse square root of the overlap matrix is formed, with numerical stability enhanced by the elimination of eigenvectors corresponding to very small eigenvalues. As a few combinations of AO basis functions may be discarded, the number of canonical-orthogonalized OSOs and MOs may be slightly smaller than the number of AOs.



3. Cholesky decomposition

When the basis set is too overcomplete, the eigendecomposition of the overlap matrix is no longer numerically stable. In this case the partial Cholesky decomposition can be used to pick a subset of basis functions that span a sufficiently complete set, see [Lehtola:2019:241102] and [Lehtola:2020:032504]. This subset can then be orthonormalized as usual; the rest of the basis functions are hidden from the calculation. The Cholesky approach allows reaching accurate energies even in the presence of significant linear dependencies [Lehtola:2020:04224].



### 5. Let us use Symmetrix orthogonalization

In [10]:
U = sp.linalg.fractional_matrix_power(S, -0.5)

### check orthonormal condition

S_prime = U @ S @ U
print(f'is S_prime orthonormal: {np.allclose(S_prime, np.eye(S_prime.shape[0]))}')

is S_prime orthonormal: True


This is good, as we can now think about diagonalization!


The drawback of this scheme is that we would now have to either re-compute the ERI and core Hamiltonian tensors in the newly orthogonal AO basis, or transform them using our $U$ matrix (both would be overly costly, especially transforming the ERI)

However we can directly subsitute $C = U C_{prime}$ into our Roothan equations!:

$$FC = S C \epsilon$$
$$F U C_{prime} = S U C_{prime} \epsilon$$

now apply $U^{\dagger}$ on both sides

$$(U^{\dagger}F U) C_{prime} = (U^{\dagger} S U) C_{prime} \epsilon$$
$$F_{prime} C_{prime} = \mathcal{I} C_{prime} \epsilon$$

$$F_{prime} C_{prime} = C_{prime} \epsilon$$


We have a standard eigenvalue equation now!!!

We can solve for the transformed orbtial coefficient matrix $C_{prime}$ by diagonalizing the transformed Fock matrix $F_{prime}$...

Then simply transform  $C_{prime}$ back to AO basis using  $C = U C_{prime}$  ( as started in the beginning)

### 6. But how do we start?

In order to find the Fock matrix we need the orbital coefficient matrix $C$, but in order to compute $C$ we need $F$...

$$F_{\mu\nu} = H_{\mu\nu} + 2 (\mu\nu|\lambda \sigma)C_{\sigma i} C_{\lambda i} -  (\mu\lambda|\nu \sigma)C_{\sigma i} C_{\lambda i}$$

We don't know $F$ or $C$... to begin we therefore GUESS the Fock matrix to obtain an initial $C$ matrix

Often a logical starting point to guessing the Fock matrix, is to start from the only part that doesn't depend on the $C$ matrix (as seen in the eq. above this is the Core Hamitlonian part!)

1. start with $F = H_{core}$ (APPROXIMATION)
2. Diagonalize transformed Fock matrix $U^{\dagger}F U)$
3. This gives initial C_prime
4. transform back to AO basis

In [11]:
F_guess = H_core

F_prime = U.conj().T @ H_core @ U

epsilon, C_prime = np.linalg.eigh(F_prime)

# transofrm C_prime back to AO basis
C = U @ C_prime

# get occupied orbitals
C_occ = C[:, :n_docc]

# build density matrix from occupied orbitals!
D = C_occ @ C_occ.T
# D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)

### 7. Perform SCF routine

1. Build Fock matrix
    - build coulomb matrix $J$
    - build exchange matrix $K$  
    - $F = H_{core} + 2J - K$
2. Find RHF energy (Szabo pg 150 (pg 134 has similar eq BUT in MO basis!)

$$E_{0} = \frac{1}{2} \sum_{\mu} \sum_{\nu} D_{\mu \nu} (H_{core} + F_{\mu \nu})$$

    - check convergence
    - if true: Break
    - else: go to 3.
3. Compute new orbital guess
    - Transform F to orthonormal AO basis
    - Diagonalize F_prime to give epsilon and C_prime
    - transform C_prime back to AO basis
    - Get new Density matrix from occupied orbitals of C

In [12]:
#alg

max_iter = 100

HF_energy =0
E_previous = 0
E_tol = 1e-6

for i in range(max_iter+1):
    
    # Build Fock Matrix
    J_mat = np.einsum('mvls,ls -> mv', eri_ao, D, optimize=True)
    K_mat = np.einsum('mlvs,ls -> mv', eri_ao, D, optimize=True)
    
    Fock = H_core + 2*J_mat - K_mat
    
    ### find RHF energy
    HF_energy = np.einsum('pq,pq ->', D, (Fock+H_core), optimize=True) + full_system_mol.energy_nuc()
#     HF_energy =np.trace(H_core @ D) + np.trace(Fock @ D) + full_system_mol.energy_nuc()
    # (no 0.5 here as included in D mat natively!)
    
    ### check convergence
    if np.abs(HF_energy-E_previous)<E_tol:
        break
        
        
    # if not convereged store old result
    E_previous = HF_energy
    
    ## compute new orbital guess
    F_prime = U.conj().T @ Fock @ U

    epsilon, C_prime = np.linalg.eigh(F_prime)

    # transofrm C_prime back to AO basis
    C = U @ C_prime

    # get occupied orbitals
    C_occ = C[:, :n_docc]

    # build density matrix from occupied orbitals!
#     D = C_occ @ C_occ.T
    D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)
    
    if i==max_iter:
        raise ValueError('Maximum number of SCF iterations exceeded')
        
print(f'final RHF SCF energy {HF_energy}')

final RHF SCF energy -75.98979522645166


In [13]:
##### check FC = SCe

# Build Fock Matrix
J_mat = np.einsum('mvls,ls -> mv', eri_ao, D, optimize=True)
K_mat = np.einsum('mlvs,ls -> mv', eri_ao, D, optimize=True)

Fock = H_core + 2*J_mat - K_mat

np.allclose(Fock@C[:,0],epsilon[0]*(S@C)[:,0])

False

# Compare to PySCF result!

In [14]:
full_system_scf = scf.RHF(full_system_mol)
full_system_scf.verbose=1
full_system_scf.max_memory= 8_000
full_system_scf.conv_tol = 1e-6

E_HF_Pyscf = full_system_scf.kernel()

In [15]:
print(f'difference in Results: {np.abs(E_HF_Pyscf - HF_energy) : 0.9f}')

difference in Results:  0.000000558


In [16]:
h_ij = np.einsum('mi,vj,mv->ij', C.conj().T, C, H_core, optimize=True)
h_ij = C @ C.conj().T @ H_core
# h_ij = np.einsum('pm,mv,vq->pq', C, H_core,C, optimize=True)

# print(np.sum(np.diag(h_ij)[:n_docc]))
O1 = np.einsum('ii->', h_ij[:n_docc, :n_docc], optimize=True)
O1

array(-62.8174697)

In [17]:
np.einsum('mv,vm ->', D, H_core, optimize=True)

array(-60.47900108)

In [18]:
# Note certain properties!

In [19]:
# By definition cannonical HF MOs are eigenfucntions of the Fock operator

# Build Fock Matrix in AO basis
J_mat = np.einsum('mvls,ls -> mv', eri_ao, D, optimize=True)
K_mat = np.einsum('mlvs,ls -> mv', eri_ao, D, optimize=True)
Fock = H_core + 2*J_mat - K_mat

# convert to MO basis
F_MO = C@C.T @ Fock

# check hartree-Fock MOs are eigenfunctions of Fock matrix!!!
for MO_ind in range(C.shape[1]):
    eigenvec_index=MO_ind
    print(np.allclose(F_MO @ C[:,eigenvec_index] , C[:,eigenvec_index]*epsilon[eigenvec_index]))

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


# IMPORTANT notes

PG 134 and 164 Szabo

Remember that HF theroy give use optimized molecular orbitals: $\{ \psi_{i} \}$

where:

$$\psi_{i} = \sum_{\mu=1}^{K} C_{\mu i} \phi_{mu}$$

- note the set $\{ \phi_{i}  | i = 1, 2 ... K\}$ are the known basis functions!


Hence the columns of $C$ from HF calculation give the molecular orbitals!

Now we often write the solution to our problem using Slater determinants - that contain molecular orbitals (NOT atomic orbs)!

$$| \Psi^{HF}> = | \psi_{1} \bar{\psi_{1}} \: \: \psi_{2} \bar{\psi_{2}} \: \: ... \: \: \psi_{N/2} \bar{\psi_{N/2}}>$$

Therefore we need to convert to the MO basis!

In [20]:
print('double occupancy =', n_docc)

HF_state = C[:, :n_docc]
HF_state.shape # aka first N/2 filled psi_i

double occupancy = 5


(24, 5)

Note $\psi_{i}$ is a molecular orbital! (not atomic!)

SO the 1e- integrals become:

$$h_{ij} = (\psi_{i} |h| \psi_{j}) = \sum_{\mu} \sum_{\nu} C^{*}_{\mu i} C_{\nu j} F_{\mu \nu}^{\text{AO basis}}$$

In [33]:
# transform 1e- ints to MO basis!

# convert to MO basis
h_ij =  C.conj().T @ Fock @ C
h_ij2 = np.einsum('mi,vj, mv->ij', C.conj(), C, Fock, optimize=True)

print(np.allclose(h_ij, h_ij2))

print(np.allclose(np.diag(h_ij), epsilon))

True
False


In [22]:
np.around(h_ij, 4) # should be diagonal, with energies as entries! see Szabo pg 165

array([[-2.0576e+01,  1.0000e-04,  0.0000e+00, -0.0000e+00, -0.0000e+00,
         0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00, -0.0000e+00,
        -0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00, -0.0000e+00,
         0.0000e+00,  1.0000e-04, -0.0000e+00, -0.0000e+00,  0.0000e+00,
        -0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00],
       [ 1.0000e-04, -1.2780e+00,  0.0000e+00,  2.0000e-04,  0.0000e+00,
        -2.0000e-04,  0.0000e+00,  0.0000e+00, -1.0000e-04,  0.0000e+00,
        -0.0000e+00,  0.0000e+00, -0.0000e+00, -1.0000e-04,  0.0000e+00,
         0.0000e+00, -3.0000e-04,  0.0000e+00,  1.0000e-04, -0.0000e+00,
         0.0000e+00,  0.0000e+00, -0.0000e+00, -0.0000e+00],
       [-0.0000e+00,  0.0000e+00, -6.3020e-01, -0.0000e+00, -0.0000e+00,
        -0.0000e+00, -2.0000e-04,  2.0000e-04, -0.0000e+00,  0.0000e+00,
         0.0000e+00,  3.0000e-04, -0.0000e+00, -0.0000e+00, -0.0000e+00,
        -0.0000e+00,  0.0000e+00, -1.0000e-04,  0.0000e+00, -0.0000e+00,
  

In [23]:
epsilon

array([-20.57291124,  -1.27694091,  -0.62948944,  -0.54113768,
        -0.48589458,   0.15774614,   0.22964136,   0.70482856,
         0.74469901,   1.17122686,   1.18693932,   1.26844382,
         1.45100902,   1.47415861,   1.6585867 ,   1.80429372,
         1.89188971,   2.1494088 ,   2.20052441,   3.17325002,
         3.21033819,   3.32871941,   3.72193771,   3.98608357])

Szabo pg 150!


$$<O_{1}> = <\Psi_{i} |O_{1} |\Psi_{i}> = \sum_{a}^{N/2}  (\psi_{a} |h| \psi_{a}) =  \sum_{\mu} \sum_{\nu} C^{*}_{\mu i} C_{\nu j} F_{\mu \nu}^{\text{AO basis}}$$

In [24]:
# Szabo pg 150!

AA = np.einsum('ii->', h_ij[:n_docc, :n_docc], optimize=True) # need 0.5 as double counting otherwise
BB = np.einsum('pq,pq ->', D, Fock, optimize=True)
print((AA, BB))
np.isclose(AA, BB)

(array(-23.51316063), array(-23.51316063))


True

notice how MO basis is much easier to evaluate $<O_{1}>$... just use Slater rules (which turns out to be diagonal elements of hij)! 

Note $\psi_{i}$ is a molecular orbital! (not atomic!)

SO the 2e- integrals become:

$$(\psi_{i} \psi_{j}|\psi_{k} \psi_{l}) = \sum_{\mu} \sum_{\nu} \sum_{\lambda} \sum_{\sigma} C^{*}_{\mu i} C_{\nu j} C^{*}_{\lambda k} C_{\sigma l} (\mu \nu | \lambda \sigma )$$

In [34]:
# transform 2e- ints to MO basis!

# Cocc = C[:, :n_docc] # occupied
# Cvirt = C[:, n_docc:] # unoccupied

# tmp = np.einsum('pi,pqrs->iqrs', Cocc, eri_ao, optimize=True)
# tmp = np.einsum('qa,iqrs->iars', Cvirt, tmp, optimize=True)
# tmp = np.einsum('iars,rj->iajs', tmp, Cocc, optimize=True)
# ERI_mo_basis = np.einsum('iajs,sb->iajb', tmp, Cvirt, optimize=True)
# mo_ints.shape

tmp = np.einsum('pi,pqrs->iqrs', C, eri_ao, optimize=True)
tmp = np.einsum('qa,iqrs->iars', C, tmp, optimize=True)
tmp = np.einsum('iars,rj->iajs', tmp, C, optimize=True)
ERI_mo_basis = np.einsum('iajs,sb->iajb', tmp, C, optimize=True)
del tmp
ERI_mo_basis.shape

(24, 24, 24, 24)

In [35]:
pyscf_mo_ints = ao2mo.kernel(full_system_mol, full_system_scf.mo_coeff)#, aosym=1)

# Convert the 2e integrals (in Chemist’s notation) 
pyscf_mo_ints = ao2mo.restore(1, pyscf_mo_ints, n_bas_ft)


# pyscf_mo_ints = mo_ints.transpose(0,2,3,1) #<- converts to physists notation if needed
pyscf_mo_ints.shape

(24, 24, 24, 24)

In [49]:
np.allclose(ERI_mo_basis, pyscf_mo_ints)

False

In [50]:
np.where(np.around(ERI_mo_basis, 3)!= np.around(pyscf_mo_ints, 3))

(array([ 0,  0,  0, ..., 23, 23, 23]),
 array([ 0,  0,  0, ..., 23, 23, 23]),
 array([ 0,  0,  0, ..., 23, 23, 23]),
 array([ 8,  9, 13, ...,  6, 15, 17]))

In [51]:
ERI_mo_basis[0,0,0,8]-pyscf_mo_ints[0,0,0,8]

-0.20966332799391596

This is 4D array for electron repulsion integrals in the MO representations...


Such MO integrals are required for all electron correlation methods. The two-electron AO integrals are the most numerous and the above equation appears to involve a computational effect proportional to M^8 (M^4 AO integrals each multiplied by four sets of M basis MO coefficients). However, by performing the transformation one index at a time, the computational effort can be reduced to M^5.

Each  step  now  only  involves  multiplication  of  M^4 basis integrals  with  M basis coefficients, i.e. the M^8 basis dependence is reduced to four M^5 operations. In the large basis set limit, all electron correlation methods formally scale as at least M 5basis, since this is the scaling for the AO to MO integral transformation. The transformation is an example of a “rotation” of the “coordinate” system consisting of the AOs, to one where the Fock operator is diagonal, the MOs (see Section 16.2). The diagonal system allows a much more compact representation  of  the  matrix  elements  needed  for  the  electron  correlation treatment. The coordinate change is also known as a four index transformation, since it involves four indices associated with the basis functions.

In [63]:
## szabo pg 73! (eq 2.111) [may need eq 2.176 pg 84!]

hcore_ij =  C.conj().T @ H_core @ C # <psi H psi> # NOT FOCK
# hcore_ij = np.einsum('mi,vj, mv->ij', C.conj(), C, H_core, optimize=True)

E = 2*np.einsum('ii->', hcore_ij[:n_docc, :n_docc], optimize=True)

two_J = 2*np.einsum('iijj->', pyscf_mo_ints[:n_docc, :n_docc, :n_docc, :n_docc], optimize=True)
K = np.einsum('ijji->', pyscf_mo_ints[:n_docc, :n_docc, :n_docc, :n_docc], optimize=True)
E+= (two_J-K)

E+= full_system_mol.energy_nuc()

E

-75.98437771164191

In [64]:
np.abs(E_HF_Pyscf - E)

0.005418072420098952

In [56]:
## szabo pg 73! (eq 2.111) [may need eq 2.176 pg 84!]

hcore_ij =  C.conj().T @ H_core @ C # <psi H psi> # NOT FOCK

E = 0
for i in range(n_docc):
    E+= 2*hcore_ij[i,i]

for i in range(n_docc):
    for j in range(n_docc):
        E+= (2*pyscf_mo_ints[i,i,j,j] - pyscf_mo_ints[i,j,j,i])

E + full_system_mol.energy_nuc()

-75.98437771164191

In [54]:
E_HF_Pyscf

-75.98979578406201

In [31]:
fadsfasdf

NameError: name 'fadsfasdf' is not defined

In [ ]:
## szabo pg 73! (eq 2.111) [may need eq 2.176 pg 84!]
E = 0
for i in range(n_docc):
    E+= h_ij[i,i]

for i in range(n_docc):
    for j in range(i, n_docc):
        E+= 0.5*(pyscf_mo_ints[i,i,j,j] - pyscf_mo_ints[i,j,j,i])

E + full_system_mol.energy_nuc()

In [ ]:
JJ=0
for i in range(n_docc):
    for j in range(n_docc):
        
        JJ += pyscf_mo_ints[i,i, j,j]
JJ

In [ ]:
pyscf_mo_ints[0,0,0,0]

In [ ]:
np.einsum('pq,pq ->', D, J_mat, optimize=True)

np.einsum('iijj ->', pyscf_mo_ints, optimize=True) # 

In [ ]:
np.einsum('iijj ->', pyscf_mo_ints[:N, :N, :N, :N], optimize=True) # [:n_docc, :n_docc, :n_docc, :n_docc]

In [ ]:
J_MO = np.einsum('iijj -> ij', pyscf_mo_ints, optimize=True) # [:n_docc, :n_docc, :n_docc, :n_docc]

J_mat = np.einsum('mvls,ls -> mv', eri_ao, D, optimize=True)
np.einsum('pq,pq ->', D, J_mat, optimize=True)

# NEXT convert the spatial MOs to spin MOs 

In [ ]:
n_qubits = 2* h_ij.shape[0]

one_body_terms = np.zeros((n_qubits, n_qubits))
two_body_terms = np.zeros((n_qubits, n_qubits, n_qubits, n_qubits))

for p in range(n_qubits//2):
    for q in range(n_qubits//2):
        
        # populate 1-body terms (must have same spin, otherwise orthogonal)
        one_body_terms[2*p, 2*q] = h_ij[p,q] # spin UP
        one_body_terms[(2*p + 1), (2*q +1)] = h_ij[p,q] # spin DOWN
        
        # continue 2-body terms
        
        for r in range(n_qubits//2):
            for s in range(n_qubits//2):
                
                ### mixed spin                
                two_body_terms[2*p, (2*q +1) , (2*r + 1), 2*s] = mo_ints[p,q,r,s] # up down down up
                two_body_terms[(2*p+1), 2*q , 2*r, (2*s +1)] = mo_ints[p,q,r,s] # down up up down
                # other mixed terms go to zero!
                
                ### SAME spin                
                two_body_terms[2*p, 2*q , 2*r, 2*s] = mo_ints[p,q,r,s] # up up up up
                two_body_terms[(2*p+1), (2*q +1) , (2*r + 1), (2*s +1)] = mo_ints[p,q,r,s] # down down down down
                
                

In [ ]:
J_mat = np.einsum('mvls,ls -> mv', eri_ao, D, optimize=True)
K_mat = np.einsum('mlvs,ls -> mv', eri_ao, D, optimize=True)

In [ ]:
# diagonal of one_body_term should be MO energies

np.diag(one_body_terms) # spin up, spin down ordering! (MO energies!)

In [ ]:
two_body_terms.shape

In [ ]:
J_mat = np.einsum('mvls,ls -> mv', eri_ao, D, optimize=True)
K_mat = np.einsum('mlvs,ls -> mv', eri_ao, D, optimize=True)
    

print(np.einsum('pq,pq ->', D, J_mat, optimize=True))
np.einsum('iijj->', two_body_terms[:N, :N, :N:, :N])

In [ ]:
EE = 0
for i in range(N):
    for j in range(N):
#         print(two_body_terms[i,i,j,j])
#         print(two_body_terms[i,j,i,j])
        EE += two_body_terms[i,i,j,j] - two_body_terms[i,j,i,j]
    
EE

In [ ]:
N = full_system_mol.nelectron
E_HF_MO = (np.einsum('ii->', one_body_terms[:N, :N], optimize=True)
           + 2*np.einsum('iijj->', two_body_terms[:N, :N, :N, :N], optimize=True)
           - np.einsum('ijij->', two_body_terms[:N, :N, :N, :N], optimize=True)
           +  full_system_mol.energy_nuc()
          )
E_HF_MO

In [ ]:
C_occ.shape

In [ ]:
one_body_terms.shape

D.shape

In [ ]:
full_system_mol.energy_nuc()

In [ ]:

CC = C[:, :n_docc]

J_mat = np.einsum('mvls,ls -> mv', ERI_mo_basis, D, optimize=True)
K_mat = np.einsum('mlvs,ls -> mv', ERI_mo_basis, D, optimize=True)

HF_energy =(np.trace(H_core @ D) 
            +  2*np.einsum('ij->', J_mat) 
            -  np.einsum('ij->', K_mat))
HF_energy

In MO basis we have:

$$E_{0} = <\psi_{0}| H_{full} | \psi_{0}> = 2 \sum_{i} (i |h|i) + \sum_{i}\sum_{j} 2(ii|jj) - 2(ij|ji)$$
$$E_{0} = 2 \sum_{i}^{N/2} h_{ii} +  \sum_{i}^{N/2} \sum_{j}^{N/2} 2 J_{ij} - K_{ij}$$

In [ ]:
J_mat = np.einsum('mvls,ls -> mv', eri_ao, D, optimize=True)
K_mat = np.einsum('mlvs,ls -> mv', eri_ao, D, optimize=True)

E_MO_HF = (2*np.einsum('ii->', h_ij[:n_docc, :n_docc])
           + 2*np.einsum('ij->', J_mat[:n_docc, :n_docc]) 
           - np.einsum('ij->', K_mat[:n_docc, :n_docc])
           + full_system_mol.energy_nuc())  
E_MO_HF

In [ ]:
D_MO = C.T @C 

J_mat_MO = np.einsum('mvls,ls -> mv', ERI_mo_basis, D_MO, optimize=True)
K_mat_MO = np.einsum('mlvs,ls -> mv', ERI_mo_basis, D_MO, optimize=True)

E_MO_HF = (2*np.einsum('ii->', h_ij[:n_docc, :n_docc]) + 
           2*np.einsum('ij->', J_mat_MO[:n_docc, :n_docc]) - 
           np.einsum('ij->', K_mat_MO[:n_docc, :n_docc])) 
E_MO_HF

In [ ]:
# D_MO = 2 * D@S

J_mat = np.einsum('mvls,ls -> mv', eri_ao, D, optimize=True)
K_mat = np.einsum('mlvs,ls -> mv', eri_ao, D, optimize=True)

D_occ = C[:, :n_docc] @ (C[:, :n_docc]).T
# D_occ = C @ C.T

J_mat = D_occ @ J_mat
K_mat = D_occ @ K_mat


twoJ_K = 2*J_mat-K_mat

E_MO_HF = 2*np.einsum('ii->', h_ij[:n_docc, :n_docc]) + np.einsum('ij->', twoJ_K[:n_docc, :n_docc])
E_MO_HF

In [ ]:
TODO: find density matrix in MO basis (think it was in one of the many opened tabs!)

In [ ]:
# D_MO = 2 * D@S

J_mat = np.einsum('mvls,ls -> mv', ERI_mo_basis, D, optimize=True)
K_mat = np.einsum('mlvs,ls -> mv', ERI_mo_basis, D, optimize=True)
twoJ_K = 2*J_mat-K_mat

E_MO_HF = 2*np.einsum('ii->', h_ij[:n_docc, :n_docc]) + np.einsum('ij->', twoJ_K[:n_docc, :n_docc]) + full_system_mol.energy_nuc()
E_MO_HF

In [ ]:
D_MO = 2* D @ S

np.trace(D_MO)

In [ ]:
 C_occ @ C_occ.T

In [ ]:
h_ij = (C @ C.conj().T @ H_core)#[:n_docc, :n_docc]

J_mat_MO = np.einsum('mvls,ls -> mv', ERI_mo_basis, D_occ, optimize=True)
K_mat_MO = np.einsum('mlvs,ls -> mv', ERI_mo_basis, D_occ, optimize=True)

twoJ_K = 2*J_mat_MO-K_mat_MO

E_MO_HF = 2*np.einsum('ii->', h_ij[:n_docc, :n_docc]) + np.einsum('ij->', twoJ_K[:n_docc, :n_docc]) + full_system_mol.energy_nuc()
E_MO_HF

In [ ]:
Fock@C[:,0]

In [ ]:
epsilon[0]*C[:,0]

In [ ]:
epsilon[2]

In [ ]:
F_MO[2,2]

In [ ]:
DD_MO =   D 

J_mat_MO = np.einsum('mvls,ls -> mv', ERI_mo_basis, DD_MO, optimize=True)
K_mat_MO = np.einsum('mlvs,ls -> mv', ERI_mo_basis, DD_MO, optimize=True)

twoJ_K = 2*J_mat_MO-K_mat_MO

E_MO_HF = 2*np.einsum('ii->', h_ij[:n_docc, :n_docc]) + np.einsum('ij->', twoJ_K[:n_docc, :n_docc])
E_MO_HF

In [ ]:
D_MO = C_occ @ C_occ.conj().T
np.allclose(D, D_MO)

In [ ]:
D_MO = 2* C_occ @ C_occ.conj().T

In [ ]:
ao2mo.restore(1, eri_ao, n_bas_ft).shape